# Constelation

In [ ]:
import os
import requests
import zipfile

In [ ]:
# populate this with url to your data
data_url = "****"
destination = os.path.join(os.getcwd(), "datasets/constelation.csv")

In [ ]:
def get_file(url, path):
    if os.path.exists(path):
        print("Skipping download {} already exists.".format(path))
        return None
    print("Downloading {} to {} ...".format(url, path))
    directory, file = os.path.split(os.path.abspath(path))
    if not os.path.exists(directory):
        os.mkdir(directory)
    r = requests.get(url, allow_redirects=True, verify=False)
    with open(path, "wb") as f:
        f.write(r.content)
    print("Complete")
        
def unzip(path, dest):
    if os.path.exists(dest):
        print("Skipping unzip as {} already exists.".format(dest))
        return None
    os.mkdir(dest)
    with zipfile.ZipFile(path, "r") as zip_ref:
        zip_ref.extractall(dest)

In [ ]:
get_file(data_url, destination)